# Predict household income from satellite imagery data

First pass.

General ML pipeline steps:
1. Import data
2. Split data into test/train sets
3. Preprocess test/train sets separately
4. Generate features from data
5. For each regressor-hyperparameter combination:
    - Train regressor with given hyperparameters and training data and labels
    - Generate predicted labels for test data with trained regressor
    - Evaluate regressor-hyperparameter performance against actual test labels and get $R^2$
6. Explore best-performing models

In [50]:
import os
import math
import pickle
import numpy as np
import pandas as pd 

from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR
from sklearn.svm import LinearSVC
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, RandomForestClassifier

from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import recall_score

import matplotlib.pyplot as plt

# Import configuration file
import config as cf

# Display options 
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = -1

# Turn off big pink warnings
import warnings
warnings.filterwarnings('ignore')

# Data file path 
final_data_file_path = "/Users/robmarty/Dropbox/World Bank/IEs/Pakistan Poverty Estimation from Satellites/Data/FinalData"
#


In [51]:
# Test grid to make sure everything works - limited models and parameters
# 'BaggingClassifier'
GRID_TEST_CLASS = {
    'regressors': ['LinearSVC','SVC', 'DecisionTreeClassifier',
                    'GradientBoostingClassifier', 'RandomForestClassifier'],
    'LinearSVC': [
        {'penalty': penalty, 'C': C, 'loss': loss, 'max_iter': max_iter,
        'random_state': 0} \
        for penalty in ('l2', ) \
        for C in (1e-2,1,2) \
        for loss in ('epsilon_insensitive','squared_hinge', ) \
        for max_iter in (1e1, )
    ],
    'SVC': [
        {'kernel': kernel, 'C': C, 'class_weight': class_weight,         
        'random_state': 0} \
        for C in (1e-2,1,2) \
        for class_weight in (None, 'balanced',) \
        for kernel in ('linear','poly','rbf','sigmoid', ) \
    ],
    'DecisionTreeClassifier': [
        {'criterion': criterion, 'splitter': splitter, 'max_depth': max_depth,
        'max_features': max_features, 'random_state': 0} \
        for criterion in ('gini', ) \
        for splitter in ('best', ) \
        for max_depth in (1,2,3,4, 5, 10, 20, 30, 50, 70, 100, ) \
        for max_features in ('sqrt', ) \
    ],
    #'BaggingClassifier': [
    #    {'n_estimators': n_estimators, 'max_features': max_features,
    #    'random_state': 0, 'n_jobs': -1} \
    #    for n_estimators in (10, 100, 1000,) \
    #    for max_features in (0.1, 0.2, 0.3,0.4, 0.5, 1.0,)
    #],
    'RandomForestClassifier': [
        {'n_estimators': n_estimators, 'criterion': criterion,
        'max_depth': max_depth, 'max_features': max_features, 'n_jobs': -1,
        'random_state': 0} \
        for n_estimators in (5, 10, 100, 1000, 5000) \
        for criterion in ('gini', ) \
        for max_depth in (1,2,3,4,5,6,7,8,9,10, ) \
        for max_features in ('sqrt','log2',None, )
    ],
    'GradientBoostingClassifier': [
        {'loss': loss, 'learning_rate': rate, 'n_estimators': n_estimators,
        'criterion': criterion, 'max_features': max_features,
        'random_state': 0} \
        for loss in ('deviance', ) \
        for rate in (1e-4, )
        for n_estimators in (100, ) \
        for criterion in ('friedman_mse', ) \
        for max_features in ('sqrt', ) \
    ]
}

## 1. Import data and drop "future" rows

In [52]:
#### Predict Changes
#DATA_PATH = os.path.join(final_data_file_path, 'BISP','Merged Datasets', 'bisp_socioeconomic_satellite_firstdiff_r13.csv')
#DATA_PATH = os.path.join('/Users/robmarty/Desktop/', 'bisp_socioeconomic_satellite_firstdiff_r13.csv')

#### Predict Levels
DATA_PATH = os.path.join(final_data_file_path, 'BISP','Merged Datasets', 'bisp_socioeconomic_satellite_panel_full_satPovNAsRemoved.csv')

df = pd.read_csv(DATA_PATH)
df.shape

(13747, 500)

In [53]:
df.head()

,uid,survey_round,province,psu,locality,period,treatment,panel,present11,present13,present16,hh_size,income_last_month_N_NAs,income_last_month,income_last_year_N_NAs,income_last_year,pscores,year,b1_buff_0.1km_mean,b2_buff_0.1km_mean,b3_buff_0.1km_mean,b4_buff_0.1km_mean,b5_buff_0.1km_mean,b6_buff_0.1km_mean,b7_buff_0.1km_mean,b12_buff_0.1km_mean,b13_buff_0.1km_mean,b14_buff_0.1km_mean,b15_buff_0.1km_mean,b16_buff_0.1km_mean,b17_buff_0.1km_mean,b23_buff_0.1km_mean,b24_buff_0.1km_mean,b25_buff_0.1km_mean,b26_buff_0.1km_mean,b27_buff_0.1km_mean,b34_buff_0.1km_mean,b35_buff_0.1km_mean,b36_buff_0.1km_mean,b37_buff_0.1km_mean,b45_buff_0.1km_mean,b46_buff_0.1km_mean,b47_buff_0.1km_mean,b56_buff_0.1km_mean,b57_buff_0.1km_mean,b67_buff_0.1km_mean,b1_buff_0.5km_mean,b2_buff_0.5km_mean,b3_buff_0.5km_mean,b4_buff_0.5km_mean,b5_buff_0.5km_mean,b6_buff_0.5km_mean,b7_buff_0.5km_mean,b12_buff_0.5km_mean,b13_buff_0.5km_mean,b14_buff_0.5km_mean,b15_buff_0.5km_mean,b16_buff_0.5km_mean,b17_buff_0.5km_mean,b23_buff_0.5km_mean,b24_buff_0.5km_mean,b25_buff_0.5km_mean,b26_buff_0.5km_mean,b27_buff_0.5km_mean,b34_buff_0.5km_mean,b35_buff_0.5km_mean,b36_buff_0.5km_mean,b37_buff_0.5km_mean,b45_buff_0.5km_mean,b46_buff_0.5km_mean,b47_buff_0.5km_mean,b56_buff_0.5km_mean,b57_buff_0.5km_mean,b67_buff_0.5km_mean,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1.5km_mean,b2_buff_1.5km_mean,b3_buff_1.5km_mean,b4_buff_1.5km_mean,b5_buff_1.5km_mean,b6_buff_1.5km_mean,b7_buff_1.5km_mean,b12_buff_1.5km_mean,b13_buff_1.5km_mean,b14_buff_1.5km_mean,b15_buff_1.5km_mean,b16_buff_1.5km_mean,b17_buff_1.5km_mean,b23_buff_1.5km_mean,b24_buff_1.5km_mean,b25_buff_1.5km_mean,b26_buff_1.5km_mean,b27_buff_1.5km_mean,b34_buff_1.5km_mean,b35_buff_1.5km_mean,b36_buff_1.5km_mean,b37_buff_1.5km_mean,b45_buff_1.5km_mean,b46_buff_1.5km_mean,b47_buff_1.5km_mean,b56_buff_1.5km_mean,b57_buff_1.5km_mean,b67_buff_1.5km_mean,b1_buff_2km_mean,b2_buff_2km_mean,b3_buff_2km_mean,b4_buff_2km_mean,b5_buff_2km_mean,b6_buff_2km_mean,b7_buff_2km_mean,b12_buff_2km_mean,b13_buff_2km_mean,b14_buff_2km_mean,b15_buff_2km_mean,b16_buff_2km_mean,b17_buff_2km_mean,b23_buff_2km_mean,b24_buff_2km_mean,b25_buff_2km_mean,b26_buff_2km_mean,b27_buff_2km_mean,b34_buff_2km_mean,b35_buff_2km_mean,b36_buff_2km_mean,b37_buff_2km_mean,b45_buff_2km_mean,b46_buff_2km_mean,b47_buff_2km_mean,b56_buff_2km_mean,b57_buff_2km_mean,b67_buff_2km_mean,b1_buff_0.1km_min,b2_buff_0.1km_min,b3_buff_0.1km_min,b4_buff_0.1km_min,b5_buff_0.1km_min,b6_buff_0.1km_min,b7_buff_0.1km_min,b12_buff_0.1km_min,b13_buff_0.1km_min,b14_buff_0.1km_min,b15_buff_0.1km_min,b16_buff_0.1km_min,b17_buff_0.1km_min,b23_buff_0.1km_min,b24_buff_0.1km_min,b25_buff_0.1km_min,b26_buff_0.1km_min,b27_buff_0.1km_min,b34_buff_0.1km_min,b35_buff_0.1km_min,b36_buff_0.1km_min,b37_buff_0.1km_min,b45_buff_0.1km_min,b46_buff_0.1km_min,b47_buff_0.1km_min,b56_buff_0.1km_min,b57_buff_0.1km_min,b67_buff_0.1km_min,b1_buff_0.5km_min,b2_buff_0.5km_min,b3_buff_0.5km_min,b4_buff_0.5km_min,b5_buff_0.5km_min,b6_buff_0.5km_min,b7_buff_0.5km_min,b12_buff_0.5km_min,b13_buff_0.5km_min,b14_buff_0.5km_min,b15_buff_0.5km_min,b16_buff_0.5km_min,b17_buff_0.5km_min,b23_buff_0.5km_min,b24_buff_0.5km_min,b25_buff_0.5km_min,b26_buff_0.5km_min,b27_buff_0.5km_min,b34_buff_0.5km_min,b35_buff_0.5km_min,b36_buff_0.5km_min,b37_buff_0.5km_min,b45_buff_0.5km_min,b46_buff_0.5km_min,b47_buff_0.5km_min,b56_buff_0.5km_min,b57_buff_0.5km_min,b67_buff_0.5km_min,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_bu

In [54]:
#### Restrict to Year
df = df[df['year'] == 2013]

In [55]:
#### Changes
#df['pscores_bin'] = df['pscores'] < 0

#### Levels
#df = df.loc[df['survey_round'] != 1]
df['pscores_bin'] = df['pscores'] <= (df['pscores'].median())
#df['pscores_bin'] = df['pscores_poor']

# DV as Quantiles
#df['pscores_2011'] = pd.qcut(df['pscores_2011'], 3, labels=False)
#df['pscores_2011'].value_counts()
#df['pscores_bin'] = df['pscores'] < 0

df.pscores_bin.value_counts()

True     2184
False    2184
Name: pscores_bin, dtype: int64

In [56]:
# Keep Select Columns
df_viirs = df.filter(regex='viirs').filter(regex='_2km')
df_landsat = df.filter(regex='^b').filter(regex='_1km')
df_osm = df.filter(regex='fclass').filter(regex='meters')
df_facebook = df.filter(regex='^estimate_dau')

df_y = df.filter(regex='^pscores_bin$')

df_all = df_y.join(df_viirs).join(df_landsat).join(df_osm).join(df_facebook)
df_all.head()

,pscores_bin,viirs_spatialmean_monthlymean_buff_2km,viirs_spatialmean_monthlysd_buff_2km,viirs_spatialmax_monthlymean_buff_2km,viirs_spatialmax_monthlysd_buff_2km,viirs_spatialmin_monthlymean_buff_2km,viirs_spatialmin_monthlysd_buff_2km,viirs_spatialsd_monthlymean_buff_2km,viirs_spatialsd_monthlysd_buff_2km,b1_buff_1km_mean,b2_buff_1km_mean,b3_buff_1km_mean,b4_buff_1km_mean,b5_buff_1km_mean,b6_buff_1km_mean,b7_buff_1km_mean,b12_buff_1km_mean,b13_buff_1km_mean,b14_buff_1km_mean,b15_buff_1km_mean,b16_buff_1km_mean,b17_buff_1km_mean,b23_buff_1km_mean,b24_buff_1km_mean,b25_buff_1km_mean,b26_buff_1km_mean,b27_buff_1km_mean,b34_buff_1km_mean,b35_buff_1km_mean,b36_buff_1km_mean,b37_buff_1km_mean,b45_buff_1km_mean,b46_buff_1km_mean,b47_buff_1km_mean,b56_buff_1km_mean,b57_buff_1km_mean,b67_buff_1km_mean,b1_buff_1km_min,b2_buff_1km_min,b3_buff_1km_min,b4_buff_1km_min,b5_buff_1km_min,b6_buff_1km_min,b7_buff_1km_min,b12_buff_1km_min,b13_buff_1km_min,b14_buff_1km_min,b15_buff_1km_min,b16_buff_1km_min,b17_buff_1km_min,b23_buff_1km_min,b24_buff_1km_min,b25_buff_1km_min,b26_buff_1km_min,b27_buff_1km_min,b34_buff_1km_min,b35_buff_1km_min,b36_buff_1km_min,b37_buff_1km_min,b45_buff_1km_min,b46_buff_1km_min,b47_buff_1km_min,b56_buff_1km_min,b57_buff_1km_min,b67_buff_1km_min,b1_buff_1km_max,b2_buff_1km_max,b3_buff_1km_max,b4_buff_1km_max,b5_buff_1km_max,b6_buff_1km_max,b7_buff_1km_max,b12_buff_1km_max,b13_buff_1km_max,b14_buff_1km_max,b15_buff_1km_max,b16_buff_1km_max,b17_buff_1km_max,b23_buff_1km_max,b24_buff_1km_max,b25_buff_1km_max,b26_buff_1km_max,b27_buff_1km_max,b34_buff_1km_max,b35_buff_1km_max,b36_buff_1km_max,b37_buff_1km_max,b45_buff_1km_max,b46_buff_1km_max,b47_buff_1km_max,b56_buff_1km_max,b57_buff_1km_max,b67_buff_1km_max,dist_osm_fclass_tertiary_meters,dist_osm_fclass_secondary_meters,dist_osm_fclass_residential_meters,dist_osm_fclass_trunk_meters,dist_osm_fclass_primary_meters,dist_osm_fclass_unclassified_meters,dist_osm_fclass_service_meters,dist_osm_fclass_motorway_meters,dist_osm_fclass_living_street_meters,estimate_dau_all,estimate_dau_male,estimate_dau_female
1,False,2.657307,0.475097,6.998805,2.596632,0.647473,0.198364,1.886077,0.780547,951.897734,1282.748257,1417.251598,2574.000436,2469.137711,3000.408919,1889.609384,0.148055,0.196422,0.460054,0.443503,0.518308,0.330005,0.049816,0.334803,0.316211,0.401027,0.191297,0.289821,0.270659,0.358370,0.142842,-0.020793,0.076494,-0.153327,0.097133,-0.132958,-0.227157,573.0,785.0,676.0,1464.0,1311.5,2926.5,779.0,0.156112,0.082466,0.437408,0.391881,0.672525,0.152367,-0.074606,0.301912,0.251133,0.576990,-0.003836,0.368224,0.319748,0.624705,0.07079,-0.054945,0.333106,-0.305395,0.381076,-0.254724,-0.579544,1590.5,2073.0,2364.0,3516.0,3834.0,3072.0,3435.0,0.131705,0.195600,0.377068,0.413587,0.317748,0.367028,0.065585,0.258186,0.298121,0.194169,0.247277,0.195918,0.237173,0.130243,0.184687,0.043265,-0.067395,-0.011653,-0.110339,-0.054891,0.055786,707.590360,691.084276,290.853957,301.885996,4213.933117,61.828224,1892.079096,5496.035379,445.707495,21718,16463,3696
4,True,2.802988,0.498989,8.192380,2.561652,0.750842,0.206752,2.101493,0.846114,941.063694,1268.392009,1402.777070,2547.212362,2463.117111,2998.701940,1876.871453,0.148149,0.196990,0.460442,0.447113,0.522274,0.332090,0.050309,0.335155,0.320172,0.405501,0.193459,0.289731,0.274281,0.362588,0.144556,-0.016784,0.081409,-0.151521,0.098060,-0.135080,-0.230092,573.0,785.0,676.0,1464.0,1282.0,2910.0,779.0,0.156112,0.082466,0.437408,0.382210,0.670973,0.152367,-0.074606,0.301912,0.240445,0.575101,-0.003836,0.368224,0.309499,0.622978,0.07079,-0.066278,0.330590,-0.305395,0.388359,-0.244056,-0.577663,1515.5,2163.0,2988.0,3639.0,5021.0,3072.0,4428.0,0.176023,0.326968,0.411970,0.536296,0.339292,0.490031,0.160163,0.254395,0.397829,0.173639,0.343650,0.098234,0.253839,0.013861,0.194175,0.159584,-0.084488,0.097806,-0.240825,-0.062758,0.180800,949.344371,821.615637,173.271184,135.784050,4011.442510,71.969127,2002.804245,5577.352264,254.089102,21718,16463,36

In [57]:
# Drop columns where the label is missing
#df = df.loc[~pd.isnull(df['hhinc_2011'])]

#df.shape

## 2. Split data into test/train

In [58]:
LABEL = 'pscores_bin'
TEST_SIZE = 0.2

# Separate feature sets from label sets
x_df = df_all.drop(labels=[LABEL], axis=1)
y_df = df_all[LABEL]

x_df[x_df.columns] = preprocessing.scale(x_df[x_df.columns])

# Split into test and train sets for features and labels
x_train, x_test, y_train, y_test =  train_test_split(x_df, y_df, test_size=TEST_SIZE)

In [59]:
x_train.head()
x_test.head()

# check that lengths match
print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))

print(y_train.value_counts())
print(y_test.value_counts())

True
True
False    1768
True     1726
Name: pscores_bin, dtype: int64
True     458
False    416
Name: pscores_bin, dtype: int64


### Define Training Variables

In [60]:
DAY_FEATURES = df_all.filter(regex='^b', axis=1).columns.tolist()
NIGHT_FEATURES = df_all.filter(regex='viirs', axis=1).columns.tolist()
SATELLITE_FEATURES = df_all.filter(regex='^b|viirs').columns.tolist()
NONSATELLITE_FEATURES = df_all.filter(regex='dist_osm|estimate_').columns.tolist()
ALL_FEATURES = x_df.columns.tolist()

MAIN_FEATURES = ['viirs_spatialmean_monthlymean_buff_2km',
    'viirs_spatialmean_monthlysd_buff_2km',
     'b1_buff_1km_mean',
 'b2_buff_1km_mean',
 'b3_buff_1km_mean',
 'b4_buff_1km_mean',
 'b5_buff_1km_mean',
 'b6_buff_1km_mean',
 'b7_buff_1km_mean',
 'b12_buff_1km_mean',
 'b13_buff_1km_mean',
 'b14_buff_1km_mean',
 'b15_buff_1km_mean',
 'b16_buff_1km_mean',
 'b17_buff_1km_mean',
 'b23_buff_1km_mean',
 'b24_buff_1km_mean',
 'b25_buff_1km_mean',
 'b26_buff_1km_mean',
 'b27_buff_1km_mean',
 'b34_buff_1km_mean',
 'b35_buff_1km_mean',
 'b36_buff_1km_mean',
 'b37_buff_1km_mean',
 'b45_buff_1km_mean',
 'b46_buff_1km_mean',
 'b47_buff_1km_mean',
 'b56_buff_1km_mean',
 'b57_buff_1km_mean',
 'b67_buff_1km_mean',
    'dist_osm_fclass_tertiary_meters',
 'dist_osm_fclass_secondary_meters',
 'dist_osm_fclass_residential_meters',
 'dist_osm_fclass_trunk_meters',
 'dist_osm_fclass_primary_meters',
 'dist_osm_fclass_unclassified_meters',
 'dist_osm_fclass_service_meters',
 'dist_osm_fclass_motorway_meters',
 'dist_osm_fclass_living_street_meters',
 'estimate_dau_all',
 'estimate_dau_male',
 'estimate_dau_female']

## 5. Train and Evaluate Regressors

### 5.1 Training

In [61]:
x_all = x_test.append(x_train)
y_all = y_test.append(y_train)

In [62]:
# Define a TrainedRegressor object to hold key results information
class TrainedRegressor:
    
    def __init__(self, method, params, features, regressor):
        self.method = method
        self.params = params
        self.regressor = regressor
        self.features = features
    
    def __repr__(self):
        return f'Trained {self.method} on feature set {self.features} with params {self.params}'

In [ ]:
# Use GRID_MAIN for full grid search
# parameters = cf.GRID_TEST_CLASS
parameters = GRID_TEST_CLASS

results_df = pd.DataFrame()
results_df_all = pd.DataFrame()
results_df_trainedonly_all = pd.DataFrame()

x_trainedonly_all = x_all.copy()

trained_list = []
trained_list_all = []
count = 0
# print('Training model ', end='')
for i in parameters['regressors']:
    for j in parameters[i]:
        for k in ('MAIN_FEATURES', 'DAY_FEATURES', 'NIGHT_FEATURES', 'ALL_FEATURES', 'SATELLITE_FEATURES', 'NONSATELLITE_FEATURES'):
        
            print(f'Model {count}: Training {i} on {k} with params {str(j)}')

            # A. Train Models --------------------------
            regressor = eval(i)(**j)
            
            trained = regressor.fit(x_train[eval(k)], y_train)
            trained_list.append(TrainedRegressor(i, str(j), k, trained))
            
            # B. Results -------------------------------------
            pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            pred_dict = {
                'regressor': trained_list[count].method,
                'features': trained_list[count].features,
                'params': trained_list[count].params,
                'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels),
                'average_precision_score': average_precision_score(y_test, pred_labels),
                'recall_score': recall_score(y_test, pred_labels)
            }
    
            results_df = results_df.append(pred_dict, ignore_index=True) \
                .sort_values(by='accuracy_score', ascending=False, axis=0) \
                [['regressor', 'params', 'features', 'accuracy_score','average_precision_score',
                 'recall_score']]
        
            results_df.to_csv("/Users/robmarty/Desktop/pov_results_r13.csv")
            
            x_test['y_true'] = y_test
            x_test['y_predict_' + str(count)] = pred_labels
            #x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict_r13.csv'))
            x_test.to_csv(os.path.join('/Users/robmarty/Desktop', 'pov_opm_data_with_predictions.csv'))

            
            
            
            
            
            
            
  
            # A. Train ------------------------------------
            # Initialize regressor, fit data, then append TrainedRegressor object to list
            # 1. Train Data
            #regressor = eval(i)(**j)
            #trained = regressor.fit(x_train[eval(k)], y_train)
            #trained_list.append(TrainedRegressor(i, str(j), k, trained))

            # 2. All Data
            #trained_all = trained
            #trained_list_all = trained_list

            
            #trained_all = regressor.fit(x_all[eval(k)], y_all)
            #trained_list_all.append(TrainedRegressor(i, str(j), k, trained_all))
            
            
            
            
            
            
            
            # B. Results -------------------------------------
            # 1. Trained Model on Test Data - - - - - - - - - -
            #pred_labels = trained_list[count].regressor.predict(x_test[eval(k)])

            #pred_dict = {
            #    'regressor': trained_list[count].method,
            #    'features': trained_list[count].features,
            #    'params': trained_list[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_test, y_pred=pred_labels)        
            #}
    
            #results_df = results_df.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df.to_csv("/Users/robmarty/Desktop/pov_results_r13.csv")
            
            #x_test['y_true'] = y_test
            #x_test['y_predict_' + str(count)] = pred_labels
            #x_test.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_traineddatamodel_testdatapredict_r13.csv'))
            
            
            
            
            
            
            
            # 2. Trained All Model on All Data - - - - - - - - - -
            #pred_labels_all = trained_list_all[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            #pred_dict = {
            #    'regressor': trained_list_all[count].method,
            #    'features': trained_list_all[count].features,
            #    'params': trained_list_all[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_all)        
            #}
    
            #results_df_all = results_df_all.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df_all.to_csv("/Users/robmarty/Desktop/pov_results_all_r13.csv")

            # ALL
            #x_trainedonly_all['y_true'] = y_all
            #x_trainedonly_all['y_predict_' + str(count)] = trained_list_all[count].regressor.predict(x_all[eval(k)])
            #x_trainedonly_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_alldatamodel_alldatapredict_r13.csv'))
            
            
            
            
            
            
            # 3. Trained Model on All Data - - - - - - - - - -
            #pred_labels_trainedonly_all = trained_list[count].regressor.predict(x_all[eval(k)])

            # Append results to dataframe and sort by R^2
            #pred_dict = {
            #    'regressor': trained_list[count].method,
            #    'features': trained_list[count].features,
            #    'params': trained_list[count].params,
            #    'accuracy_score': accuracy_score(y_true=y_all, y_pred=pred_labels_trainedonly_all)        
            #}
    
            #results_df_trainedonly_all = results_df_trainedonly_all.append(pred_dict, ignore_index=True) \
            #    .sort_values(by='accuracy_score', ascending=False, axis=0) \
            #    [['regressor', 'params', 'features', 'accuracy_score']]
        
            #results_df_trainedonly_all.to_csv("/Users/robmarty/Desktop/pov_results_trainedonly_all_r13.csv")

            # ALL
            #x_all['y_true'] = y_all
            #x_all['y_predict_' + str(count)] = trained_list[count].regressor.predict(x_all[eval(k)])
            #x_all.to_csv(os.path.join(final_data_file_path, 'Data with Predicted Income', 'pov_opm_data_with_predictions_testdatamodel_alldatapredict_r13.csv'))

            ####
            count += 1


Model 0: Training LinearSVC on MAIN_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 1: Training LinearSVC on DAY_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 2: Training LinearSVC on NIGHT_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 3: Training LinearSVC on ALL_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 4: Training LinearSVC on SATELLITE_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 5: Training LinearSVC on NONSATELLITE_FEATURES with params {'penalty': 'l2', 'C': 0.01, 'loss': 'epsilon_insensitive', 'max_iter': 10.0, 'random_state': 0}
Model 6: Training LinearSVC on MAIN_FEATURES with params {'penal